I referred this kernel for leaning how to use Keras.

[Keras Starter](https://www.kaggle.com/ateplyuk/keras-starter)

In [37]:
import numpy as np
import pandas as pd
import os

In [39]:
train = pd.read_csv("../input/imet-2019-fgvc6/train.csv")
labels = pd.read_csv("../input/imet-2019-fgvc6/labels.csv")
sub = pd.read_csv("../input/imet-2019-fgvc6/sample_submission.csv")

train["id"] = train.id.map(lambda x: "{}.png".format(x))
train["attribute_ids"] = train.attribute_ids.map(lambda x: x.split())
sub["id"] = sub.id.map(lambda x: "{}.png".format(x))

display(sub.head())
display(train.head())

,id,attribute_ids
0,10023b2cc4ed5f68.png,0 1 2
1,100fbe75ed8fd887.png,0 1 2
2,101b627524a04f19.png,0 1 2
3,10234480c41284c6.png,0 1 2
4,1023b0e2636dcea8.png,0 1 2


,id,attribute_ids
0,1000483014d91860.png,"[147, 616, 813]"
1,1000fe2e667721fe.png,"[51, 616, 734, 813]"
2,1001614cb89646ee.png,[776]
3,10041eb49b297c08.png,"[51, 671, 698, 813, 1092]"
4,100501c227f8beea.png,"[13, 404, 492, 903, 1093]"


In [41]:
batch_size = 32
img_size = 64
nb_epochs = 150
nb_classes = labels.shape[0]
lbls = list(map(str, range(nb_classes)))

In [47]:
%%time

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

train_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/imet-2019-fgvc6/train",
    x_col="id",
    y_col="attribute_ids",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",
    classes=lbls,
    target_size=(img_size,img_size),
    subset='training')

valid_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/imet-2019-fgvc6/train",
    x_col="id",
    y_col="attribute_ids",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",    
    classes=lbls,
    target_size=(img_size,img_size),
    subset='validation')


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(  
        dataframe=sub,
        directory = "../input/imet-2019-fgvc6/test",    
        x_col="id",
        target_size = (img_size,img_size),
        batch_size = 1,
        shuffle = False,
        class_mode = None
        )

Found 81928 images belonging to 1103 classes.
Found 27309 images belonging to 1103 classes.
Found 7443 images.
CPU times: user 5.77 s, sys: 4.6 s, total: 10.4 s
Wall time: 1min 18s


In [48]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, Flatten

vgg_conv = VGG16(weights=None, include_top=False, input_shape=(img_size, img_size, 3))
vgg_conv.load_weights('../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

model = Sequential()
model.add(vgg_conv)
 
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_8 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 1103)              1130575   
Total params: 17,943,439
Trainable params: 10,308,175
Non-trainable params: 7,635,264
_________________________________________________________________


In [49]:
from keras import optimizers

model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [50]:
history = model.fit_generator(
                    generator=train_generator,
                    steps_per_epoch=100,
                    validation_data=valid_generator,
                    validation_steps=50,
                    epochs=nb_epochs,
                    verbose=1)

Epoch 1/50
100/100 [==============================] - 39s 394ms/step - loss: 18.1495 - acc: 0.0609 - val_loss: 16.2219 - val_acc: 0.1212
Epoch 2/50
100/100 [==============================] - 37s 366ms/step - loss: 16.5119 - acc: 0.0872 - val_loss: 15.9322 - val_acc: 0.1281
Epoch 3/50
100/100 [==============================] - 37s 372ms/step - loss: 16.0891 - acc: 0.1016 - val_loss: 15.4635 - val_acc: 0.0994
Epoch 4/50
100/100 [==============================] - 37s 371ms/step - loss: 16.1413 - acc: 0.0994 - val_loss: 15.4923 - val_acc: 0.1087
Epoch 5/50
100/100 [==============================] - 35s 352ms/step - loss: 15.6467 - acc: 0.1147 - val_loss: 15.4462 - val_acc: 0.1319
Epoch 6/50
100/100 [==============================] - 36s 356ms/step - loss: 15.3330 - acc: 0.1194 - val_loss: 14.8392 - val_acc: 0.1275
Epoch 7/50
 96/100 [===========================>..] - ETA: 1s - loss: 15.2040 - acc: 0.1276

KeyboardInterrupt: 

In [ ]:
import json

with open('history.json', 'w') as f:
    json.dump(history.history, f)

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

In [ ]:
%%time

test_generator.reset()
predict=model.predict_generator(test_generator, steps = len(test_generator.filenames))

In [ ]:
predicted_class_indices = np.argmax(predict,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

sub["attribute_ids"] = predictions
sub['id'] = sub['id'].map(lambda x: str(x)[:-4])

sub.to_csv("submission.csv",index=False)
sub.head()